In [1]:
from pptx import Presentation
import json

class PowerPointProcessor:
    def __init__(self, ppt_path):
        self.ppt_path = ppt_path
        self.presentation = Presentation(ppt_path)
        self.slides_text = {}

    def process_presentation(self):
        for slide_index, slide in enumerate(self.presentation.slides, start=1):
            slide_content = []
            
            for shape in slide.shapes:
                if shape.has_text_frame:
                    text = "\n".join([paragraph.text for paragraph in shape.text_frame.paragraphs])
                    slide_content.append(text)
                elif shape.has_table:
                    table = shape.table
                    table_text = []
                    for row in table.rows:
                        row_text = [cell.text for cell in row.cells]
                        table_text.append(row_text)
                    slide_content.append(table_text)
            
            self.slides_text[slide_index] = slide_content

    def save_as_json(self, json_path):
        with open(json_path, "w") as json_file:
            json.dump(self.slides_text, json_file, ensure_ascii=False, indent=2)

    def get_slides_text(self):
        return self.slides_text

    def translate_and_replace(self, translated_json):
        for slide_index, slide in enumerate(self.presentation.slides, start=1):
            if str(slide_index) in translated_json:
                slide_content = translated_json[str(slide_index)]
                content_index = 0
                
                for shape in slide.shapes:
                    if shape.has_text_frame:
                        text = slide_content[content_index]
                        for i, paragraph in enumerate(shape.text_frame.paragraphs):
                            paragraph.text = text.split("\n")[i] if i < len(text.split("\n")) else ''
                        content_index += 1
                    elif shape.has_table:
                        table = shape.table
                        table_text = slide_content[content_index]
                        for row_index, row in enumerate(table.rows):
                            for cell_index, cell in enumerate(row.cells):
                                cell.text = table_text[row_index][cell_index] if row_index < len(table_text) and cell_index < len(table_text[row_index]) else ''
                        content_index += 1

    def save_presentation(self, new_ppt_path):
        self.presentation.save(new_ppt_path)

In [2]:
#from PowerPointProcessor import PowerPointProcessor

ppt_path = "テスト.pptx"
processor = PowerPointProcessor(ppt_path)

In [3]:
processor.process_presentation()

In [4]:
json_path = "slides_text.json"
processor.save_as_json(json_path)

In [5]:
def trans(json_data):
    def translate_text(text):
        # Dummy translation function for example
        return text+'Translated'

    def translate_item(item):
        if isinstance(item, str):
            return translate_text(item)
        elif isinstance(item, list):
            return [translate_item(sub_item) for sub_item in item]
        elif isinstance(item, dict):
            return {key: translate_item(value) for key, value in item.items()}
        else:
            return item

    return translate_item(json_data)

In [6]:
with open(json_path, "r") as json_file:
    original_json = json.load(json_file)

translated_json = trans(original_json)

processor.translate_and_replace(translated_json)

In [7]:
processor.save_presentation('テスト_翻訳2.pptx')